# Resize all input images for easier processing

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-required-libraries" data-toc-modified-id="Import-required-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import required libraries</a></span></li><li><span><a href="#Create-a-dataframe-with-images-from-all-subfolders-in-PATH_INPUT" data-toc-modified-id="Create-a-dataframe-with-images-from-all-subfolders-in-PATH_INPUT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create a dataframe with images from all subfolders in PATH_INPUT</a></span><ul class="toc-item"><li><span><a href="#Get-list-of-valid-images-in-a-path" data-toc-modified-id="Get-list-of-valid-images-in-a-path-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get list of valid images in a path</a></span></li><li><span><a href="#Place-the-list-of-images-into-a-dataframe" data-toc-modified-id="Place-the-list-of-images-into-a-dataframe-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Place the list of images into a dataframe</a></span></li><li><span><a href="#Create-output-directories-to-store-the-augmented-images" data-toc-modified-id="Create-output-directories-to-store-the-augmented-images-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create output directories to store the augmented images</a></span></li></ul></li><li><span><a href="#Resize-Images" data-toc-modified-id="Resize-Images-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Resize Images</a></span></li><li><span><a href="#Resize-images" data-toc-modified-id="Resize-images-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Resize images</a></span><ul class="toc-item"><li><span><a href="#Helper-function-for-reading,-resizing-and-saving-images-to-a-location" data-toc-modified-id="Helper-function-for-reading,-resizing-and-saving-images-to-a-location-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Helper function for reading, resizing and saving images to a location</a></span></li><li><span><a href="#Resize-the-images" data-toc-modified-id="Resize-the-images-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Resize the images</a></span></li></ul></li></ul></div>

## Import required libraries

In [3]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import imutils
import shutil

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
%matplotlib inline

In [30]:
PATH_INPUT = r'/Users/shyamravikumar/Documents/Box Sync/Final Project - Image Detection/03 Defect Images/For Testing Purpose - Priyank'
PATH_OUTPUT = r'/Users/shyamravikumar/Documents/Workspace/BAI - Project/Input Files/V2/Small/1. Resized Images'

In [31]:
# Source image size - (3672, 4896, 3)
# ASPECT_RATIO = 4896/3672

# Size for small images
HEIGHT_NEW = 306
WIDTH_NEW = 408 

# Size for large images
# HEIGHT_NEW = 612
# WIDTH_NEW = 816

## Create a dataframe with images from all subfolders in PATH_INPUT

### Get list of valid images in a path

In [32]:
def get_valid_images_in_path(path):
    """
    Return a list of valid images in a path
    
    Parameters
    ----------
        
        path: str
            Location where the images are stored
            
    Returns
    -------
        
        list of str: List of images obtained from the directory
        
    """
    from keras_preprocessing.image.utils import _iter_valid_files
    from keras_preprocessing.image.directory_iterator import DirectoryIterator
    
    list_files = list(
        _iter_valid_files(path,
                          DirectoryIterator.white_list_formats,
                          follow_links=False))
    list_files = list(map(lambda x: os.path.join(*x), list_files))
    return list_files

In [33]:
# Identify the list of available categories/classes
classes = []
for subdir in sorted(os.listdir(PATH_INPUT)):
    if os.path.isdir(os.path.join(PATH_INPUT, subdir)):
        classes.append(subdir)
classes

['defective_lav', 'defective_ldl', 'non_defective', 'non_defective_bubbles']

### Place the list of images into a dataframe

In [34]:
list_dataframes = []
for category in classes:
    path_category = os.path.join(PATH_INPUT, category)
    list_images = get_valid_images_in_path(path_category)
    list_dataframes.append(
        pd.DataFrame({
            'path': list_images,
            'class': np.repeat(category, len(list_images))
        })
    )

In [35]:
data_images = pd.concat(list_dataframes)
data_images['filename'] = data_images['path'].apply(os.path.basename)

In [36]:
# Create a column to identify the output path
data_images['path_output'] = data_images.apply(lambda x: os.path.join(PATH_OUTPUT, x['class'],x['filename']),axis=1)

In [37]:
data_images.head()

,path,class,filename,path_output
0,/Users/shyamravikumar/Documents/Box Sync/Final...,defective_lav,Sample 2 - Pattern 1 - 1.JPG,/Users/shyamravikumar/Documents/Workspace/BAI ...
1,/Users/shyamravikumar/Documents/Box Sync/Final...,defective_lav,Sample 3 - Pattern 1 - 1.JPG,/Users/shyamravikumar/Documents/Workspace/BAI ...
2,/Users/shyamravikumar/Documents/Box Sync/Final...,defective_lav,Sample 3 - Pattern 1 - 2.JPG,/Users/shyamravikumar/Documents/Workspace/BAI ...
3,/Users/shyamravikumar/Documents/Box Sync/Final...,defective_lav,Sample 5 - Pattern 1 - 1.JPG,/Users/shyamravikumar/Documents/Workspace/BAI ...
4,/Users/shyamravikumar/Documents/Box Sync/Final...,defective_lav,Sample 5 - Pattern 1 - 2.JPG,/Users/shyamravikumar/Documents/Workspace/BAI ...


In [38]:
data_images.iloc[0].values

array(['/Users/shyamravikumar/Documents/Box Sync/Final Project - Image Detection/03 Defect Images/For Testing Purpose - Priyank/defective_lav/Sample 2 - Pattern 1 - 1.JPG',
       'defective_lav', 'Sample 2 - Pattern 1 - 1.JPG',
       '/Users/shyamravikumar/Documents/Workspace/BAI - Project/Input Files/V2/Small/1. Resized Images/defective_lav/Sample 2 - Pattern 1 - 1.JPG'],
      dtype=object)

### Create output directories to store the augmented images

In [40]:
# Ensure that the required o/p directories (for each class) exist. If not, they will be created
for feature in classes:
    folder = os.path.join(PATH_OUTPUT, feature)
    if not os.path.exists(folder):
        os.mkdir(folder)

## Resize Images

Set the desired height and width for each image (in pixels)

## Resize images

### Helper function for reading, resizing and saving images to a location

In [41]:
def read_resize_and_save(path_old,path_new):
    img = cv2.imread(path_old)
    img = cv2.resize(img,(WIDTH_NEW,HEIGHT_NEW))
    cv2.imwrite(path_new,img)

### Resize the images

In [42]:
data_images.apply(lambda x: read_resize_and_save(x['path'],x['path_output']),axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
      ... 
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
Length: 127, dtype: object